In [5]:
from openai import OpenAI
import os
from dotenv import dotenv_values
config = dotenv_values('.env')
client = OpenAI(
    api_key = config['qwen_key'],
    base_url = config['qwen_url']
)

def get_response(prompt):
    completion = client.chat.completions.create(
        model="qwen2-7b-instruct",
        messages=[{'role': 'user', 'content': prompt}]
        )
    result = completion.choices[0].message.content
    return result

if __name__ == '__main__':
    result = get_response("你好")
    print(result)

你好！很高兴能为你服务。有什么我可以帮助你的吗？


In [1]:
from ReadLoad import read_json, write_jsonl
dev_data = read_json("dev.json")

In [6]:
#!pip install dashscope
import dashscope
from http import HTTPStatus
dashscope.api_key = config['qwen_key'] 

def get_embedding(embed_text):
    
    respond = ''
    try:
        respond = dashscope.TextEmbedding.call(
            model = dashscope.TextEmbedding.Models.text_embedding_v2,
            input = embed_text)
        
        embedding = respond.output['embeddings'][0]['embedding']
        
    except Exception as e:
        # Handle any other unexpected exceptions
        print(f"An unexpected error occurred: {e}")
    
    return embedding

In [18]:
!pip install tqdm
from tqdm import tqdm
for data in tqdm(dev_data):
    embed_text = data['答案']
    data['ans_embedding'] = get_embedding(embed_text)

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


100%|██████████| 1000/1000 [03:59<00:00,  4.17it/s]


In [21]:
 write_jsonl(dev_data,"dev_with_embedding")

In [20]:
dev_data[0]

{'ID': '5000',
 '产品名': '众安中高端医疗险（含门诊）',
 '条款': '重建手术费 指被保险人为恢复身体外观，在医生建议下于意外事故发生后12个月内进行重建手术的实际开支；或被保险人因疾病而进行乳房切除手术或乳房切除手术后12个月内进行的重建乳房手术的实际开支。在保险期间内，保险人所承担被保险人每次住院重建手术费的累计给付金额可由投保人和保险人在投保时单独约定，并在保险单中载明。',
 '问题': '什么是众安中高端医疗险中的重建手术费？',
 '答案': '重建手术费是指被保险人为恢复身体外观，在医生建议下于意外事故发生后12个月内进行重建手术的实际开支；或者是被保险人因疾病而进行乳房切除手术或乳房切除手术后12个月内进行的重建乳房手术的实际开支。',
 'embedding': [0.002810733902871633,
  -0.0023343805972385525,
  0.01560244125675682,
  0.00174080946247063,
  -0.03563422005699192,
  0.006908369929861956,
  0.026695737086369086,
  0.05099723766274992,
  -0.016201000384253886,
  0.00522242838741189,
  0.013726955990599352,
  0.003002771622943608,
  -0.019752451207403137,
  0.017038983162749776,
  -0.012928877153936598,
  -0.006075375144095207,
  0.018365789228701603,
  0.011482359262485359,
  -0.019822283105611127,
  0.05035877459341971,
  0.010744136338572312,
  0.029229637392773327,
  -0.008684095341436581,
  0.025598378685957802,
  -0.025518570802291526,
  -0.016510255933460703,
  0.0536708017

In [12]:
import numpy as np
def cosine_similarity(vec1, vec2):
    """
    Calculate the cosine similarity between two vectors.
    
    :param vec1: First vector (numpy array).
    :param vec2: Second vector (numpy array).
    :return: Cosine similarity score (float).
    """
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

In [17]:
text1 = "1000万航空意外险具有一年的默认保险期间"
text2 = "1000万航空意外险默认保险期间一年。"
vec1 = get_embedding(text1)
vec2 = get_embedding(text2)
similarity = cosine_similarity(vec1, vec2)
print(similarity)

0.9570271829028671


### 参考资料

https://dashscope.console.aliyun.com/billing